In [89]:
%pwd

'd:\\ml\\end_to_end_airbnb_machine_learning_project\\Seattle-airbnb-house-price-prediction-with-mlflow-and-aws-deployment'

In [90]:
import os
os.chdir("d:\\ml\\end_to_end_airbnb_machine_learning_project\\Seattle-airbnb-house-price-prediction-with-mlflow-and-aws-deployment")

%pwd

'd:\\ml\\end_to_end_airbnb_machine_learning_project\\Seattle-airbnb-house-price-prediction-with-mlflow-and-aws-deployment'

In [91]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth: int
    max_features: str
    random_state: int
    target_column: str  


In [92]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [93]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            max_depth = params.max_depth,
            max_features = params.max_features,
            random_state = params.random_state,
            target_column = schema.name
            
        )

        return model_trainer_config

In [94]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import os
from mlProject import logger
import joblib

In [95]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop([self.config.target_column], axis=1)
        Y_train = train_data[[self.config.target_column]].values.ravel()

        X_test = test_data.drop([self.config.target_column], axis=1)
        Y_test = test_data[[self.config.target_column]].values.ravel()

        rf = RandomForestRegressor(
            n_estimators=self.config.n_estimators,
            max_depth=self.config.max_depth,
            max_features=self.config.max_features,
            random_state=self.config.random_state
        )

        rf.fit(X_train, Y_train)

        joblib.dump(rf, os.path.join(self.config.root_dir, self.config.model_name))
        logger.info(f"Model saved at {os.path.join(self.config.root_dir, self.config.model_name)}")

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-12-04 16:43:03,226: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-04 16:43:03,230: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-04 16:43:03,233: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-04 16:43:03,233: INFO: common: created directory at: artifacts]
[2024-12-04 16:43:03,237: INFO: common: created directory at: artifacts/model_trainer]
[2024-12-04 16:43:03,765: INFO: 912053770: Model saved at artifacts/model_trainer\model.joblib]


In [96]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-12-04 16:43:03,787: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-04 16:43:03,795: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-04 16:43:03,799: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-04 16:43:03,801: INFO: common: created directory at: artifacts]
[2024-12-04 16:43:03,803: INFO: common: created directory at: artifacts/model_trainer]
[2024-12-04 16:43:04,348: INFO: 912053770: Model saved at artifacts/model_trainer\model.joblib]
